In [1]:
import os
import pandas as pd
import numpy as np
from importlib import reload

import data_wrangle
import spectral
import analysis
import __utils__

from prettytable import PrettyTable as pt

#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

from math import ceil
from collections import Counter

# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"

# Initalize Object and set constants

In [2]:
path='/mnt/c/Users/16162/Desktop/dopamine.db'
table='arsenic'

## Look at the Table

In [170]:
reload(data_wrangle)
db=data_wrangle.DataTable(path)
# db.summary() # fix column info

db.__del__()

A table must be selected for the Object to be initialized.
Available tables (5):
	initial_spectra
	micro
	macro
	da_dopac
	arsenic


OperationalError: no such table: None

# Grab data
---

+ Can remove spectra by id:  
`db.deab_list.append(#)`   
+ Can set conditions:  
    + `db.condition='WHERE col IS NOT NULL'`  
    + `db.condition='WHERE col IS value'`  
    + `db.condition='WHERE col1 IS NOT NULL AND col2 = value'`
    + `db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cortex\',\'midbrain\')'`  
    + `db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"`
+ As currently write, `raman_shifts()` should be executed twice to automatically remove problematic Raman Shifts

In [152]:
reload(data_wrangle)

#1) Initalize Table
db=data_wrangle.DataTable(path, table)

x=db.raman_shifts()
ys=db.intens()

There are 2 Ramanshift lists found in the dataset.
+----------------------------------------------+
|                   Outliers                   |
+----------------------------------------------+
| There are no outliers in the remaining data! |
+----------------------------------------------+
+----------------------------------------------+


In [137]:
#2) Generate labeling dictionaries
reload(__utils__)
reload(data_wrangle)
db=data_wrangle.DataTable(path, table)
x=db.raman_shifts()
ys=db.intens()
namedic=db.label_dict(cols=['As3','As5'])

__utils__.dict_summary(namedic)

('There are 225 entires and 9 unique values in the dictionary.',
 +-----------------------+-------------------+
 |       Dict Value      | Frequency in Dict |
 +-----------------------+-------------------+
 | 50 uM As3, 150 uM As5 |         25        |
 |      0 As3, 0 As5     |         25        |
 |  19 uM As3, 1 mM As5  |         25        |
 |    50 uM As3, 0 As5   |         25        |
 | 19 uM As3, 150 uM As5 |         25        |
 |    19 uM As3, 0 As5   |         25        |
 |    0 As3, 1 mM As5    |         25        |
 |  50 uM As3, 1 mM As5  |         25        |
 |   0 As3, 150 uM As5   |         25        |
 +-----------------------+-------------------+)

# View spectra

In [70]:
# all_samples=list(set(mydic.values()))

color_dic={
'19 uM As3, 1 mM As5':'#8200ff',
'0 As3, 150 uM As5':'#6600ff',
'0 As3, 1 mM As5':'#250094',
'19 uM As3, 0 As5':'#ff0061',
'0 As3, 0 As5':'grey',
'19 uM As3, 150 uM As5':'#da00c6',
'50 uM As3, 0 As5':'#b40000',
'50 uM As3, 1 mM As5':'#a800e7',
'50 uM As3, 150 uM As5':'#ff00ae',
}

set_dic={
  'Mix':['50 uM As3, 1 mM As5', '19 uM As3, 1 mM As5', '19 uM As3, 150 uM As5', '50 uM As3, 150 uM As5'],
  'As3':['50 uM As3, 0 As5','19 uM As3, 0 As5'],
  'As5':['0 As3, 1 mM As5','0 As3, 150 uM As5'],
  'Blank':['0 As3, 0 As5']
 }

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [169]:
#testing
reload(spectral)
db=data_wrangle.DataTable(path, table)
x=db.raman_shifts()
ys=db.intens()
namedic=db.label_dict(cols=['As3','As5'])

spec=spectral.Spectral(ys,x,namedic,color_dict=color_dic)
spec.single_plot(show_std=False)

# color_dic['19 uM As3, 1 mM As5']

There are 2 Ramanshift lists found in the dataset.
+----------------------------------------------+
|                   Outliers                   |
+----------------------------------------------+
| There are no outliers in the remaining data! |
+----------------------------------------------+
+----------------------------------------------+


In [127]:
# for group in list(set(mydic.values())):
group_name=list(set(mydic.values()))[0]
ids=[key for key, value in mydic.items() if value == group_name]
existing_ids=list(set(ids).intersection(df.index))
ave=df.loc[existing_ids].mean()
std=df.loc[existing_ids].std()
traces=[]
traces.append(go.Scatter(x=x,y=ave,name=group_name))
traces.append(go.Scatter(x=x, y=ave + std, mode='lines', name='upper bound', line=dict(color='lightgrey')))
traces.append(go.Scatter(x=x, y=ave - std, mode='lines', name='lower bound', line=dict(color='lightgrey'), fill='tonexty', opacity=0.1))
go.Figure(traces).show()

In [93]:
#Few spectra, averaged by sample group
df=ys
mydic=namedic
x=x
traces=[]
for sample_group in list(set(mydic.values())): #grab each sample group
    ids=[idx for idx, label in mydic.items() if label == sample_group] #Get the sample ids for specificed label
    existing_ids=list(set(ids).intersection(df.index)) #Filter out ids that do not occur in the DF
    # print(len(ids), len(existing_ids))
    # Get the average
    if existing_ids:
        ave=df.loc[existing_ids].mean() #Find average of all spectra with same label
        std=df.loc[existing_ids].std() #Calcuate the standard deviation
        traces.append(go.Scatter(y=ave, x=x, name=sample_group))
    else:
        print(f'No samples in DF for {sample_group}')
fig=go.Figure(traces)
fig.update_layout(template='simple_white')
fig.show()
    

In [142]:
for x in set(namedic.values()):
    print(f'\'{x}\':\'#\',',)

'19 uM As3, 1 mM As5':'#',
'0 As3, 150 uM As5':'#',
'0 As3, 1 mM As5':'#',
'19 uM As3, 0 As5':'#',
'0 As3, 0 As5':'#',
'19 uM As3, 150 uM As5':'#',
'50 uM As3, 0 As5':'#',
'50 uM As3, 1 mM As5':'#',
'50 uM As3, 150 uM As5':'#',


# Grouping data and making spectral figures

In [143]:
# all_samples=list(set(mydic.values()))

color_dic={
'19 uM As3, 1 mM As5':'#8200ff',
'0 As3, 150 uM As5':'#6600ff',
'0 As3, 1 mM As5':'#250094',
'19 uM As3, 0 As5':'#ff0061',
'0 As3, 0 As5':'grey',
'19 uM As3, 150 uM As5':'#da00c6',
'50 uM As3, 0 As5':'#b40000',
'50 uM As3, 1 mM As5':'#a800e7',
'50 uM As3, 150 uM As5':'#ff00ae',
}

set_dic={
  'Mix':['50 uM As3, 1 mM As5', '19 uM As3, 1 mM As5', '19 uM As3, 150 uM As5', '50 uM As3, 150 uM As5'],
  'As3':['50 uM As3, 0 As5','19 uM As3, 0 As5'],
  'As5':['0 As3, 1 mM As5','0 As3, 150 uM As5'],
  'Blank':['0 As3, 0 As5']
 }

In [65]:
#Grouping samples by label

group_dfs={} # Empty dictionary to store subdfs

# Iterate over the items in group_dic
for group, sample_labels in group_dic.items():
    
    # Filter the samples based on the sample label
    sample_ids=[i for i, label in namedic.items() if label in sample_labels] # Get the ids that match each group

    # Create subdfs from the sample_ids list
    subdf=ys.loc[sample_ids]

    # Add the subdf to the dictionary
    group_dfs[group]=subdf
    

# Truncate Code

In [191]:
reload(__utils__)
#This should go into spectral
def truncate(x_data, y_data, before_target=None, after_target=None):
    '''targets need to be intergers for the Raman Shift values.
    x_data = Raman Shifts as a series or list
    y_data = df where each row is a sample
    If no only 1 target is given, this should still work'''#***
    #Find indexes for truncating
    if before_target==None:
        before_idx=0
    else:
        before_idx=__utils__.find_index(x_data, before_target)
    if after_target==None:
        after_idx=len(x_data)
    else:
        after_idx=__utils__.find_index(x_data, after_target)
    print(after_idx,'FIX return error if NONE')
    res_x=x_data[before_idx:after_idx]
    res_ys=y_data.iloc[:,before_idx:after_idx]
    t1=go.Scatter(x=x_data, y=y_data.iloc[1], name='before')
    t2=go.Scatter(x=res_x, y=res_ys.iloc[1], name='after')
    traces=[t1,t2]
    fig=go.Figure(traces)
    fig.update_layout(title='Visualize Data Truncation')
    fig.show()

    return res_x, res_ys

x,y=truncate(x,ys,500,1080)

592 FIX return error if NONE


In [204]:
color_dic={
 '50 uM, 1 mM':'purple',
 '50 uM, 0':'red',
 '0, 150 uM':'blue',
 '19 uM, 1 mM':'purple',
 '0, 1 mM':'blue',
 '19 uM, 0':'red',
 '19 uM, 150 uM':'purple',
 '50 uM, 150 uM':'purple',
 '0, 0':'darkgrey'}

# PCA

In [105]:
reload(analysis)
var=analysis.multivar(ys,x,namedic, color_dict=color_dic)
# pc,a,b=var.pca_run()
# var.pca_fig('PC 1', 'PC 2')
# var.pca_fig('PC 1', 'PC 3')
# var.pca_fig('PC 2', 'PC 3')
# var.pca_3d()


In [92]:
ls=var.loadings('1','2')
for l in ls:
    l.show()


# Data Work

## Get Data From DB

___

+ `x=db.raman_shifts()`
+ `ys=db.intens()`
+ `namedic=db.name_dic()`
+ `ys=data_wrangle.map_index(ys,namedic)`   # Fix this

+ Get rid of summary prettytable in `data_info()`

# Analysis

`pio.renderers.default = "svg"`  
`var=multivar(ys.T,x,names,names)`  
`figs=var.pca_fig('PC 1','PC 2',ys.index)`  
`figs=var.pca_figs('PC 2', 'PC 3')`  
`var.pca_fig(figs)`  
`var.pca_fig`  
`var.pca_3d(color=names)`

# Update Database Find and Replace

In [31]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace
# db.find_replace('BoooTh', 'Booo', 'nanotag')

Number of samples selected: 75


[+-------------------+
 |      nanotag      |
 +-------+-----------+
 | Value | Frequency |
 +-------+-----------+
 |  Booo |     69    |
 |  NaN  |     1     |
 |  nba  |     1     |
 |   no  |     4     |
 +-------+-----------+]

# Some extras and old code

In [ ]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

#Map namedic to sample ID
tag=ys.index.map(lambda x: namedic[x][0])

In [ ]:
len(db.all_ids)
len(db.dead_list)

54

## Code i was working on a while ago

In [ ]:
df=pd.DataFrame([[1,2,3,4,5],[2,3,4,5,6],[1,2,3,4,5],[5,6,7,8,9],[1,2,3,4,5],[5,6,7,8,9]])
print(df)
uniques=set(map(tuple,df.values))
uniques=list(uniques)
for u in uniques:
    mask=np.all(df==u, axis=1)
    print(mask)
print(df[mask])
print(df)


   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9
0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool
0     True
1    False
2     True
3    False
4     True
5    False
dtype: bool
0    False
1     True
2    False
3    False
4    False
5    False
dtype: bool
   0  1  2  3  4
1  2  3  4  5  6
   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9


In [ ]:
#Update find/replace

#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
conn=sqlite3.connect('/mnt/c/Users/16162/Desktop/Brain.db')
c=conn.cursor()

#Set search conditions
date='2023-04-20'

#Col to update
col_to_update='spike2'
search='spiked2'

#Find entries per search conditions
c.execute(f'SELECT id, {col_to_update}, filename FROM sheep WHERE date = ?',(date,))
# c.execute(f'SELECT id, {col_to_update}, filename FROM sheep WHERE date = ?',(date,))
rows=c.fetchall()

#Loop through each entry
for row in rows:
    if search in row[2]:
        new='yes'
        c.execute(f'UPDATE sheep SET {col_to_update} = ? WHERE id = ?',(new,row[0],))
    # elif 'spiked2' in row[2]:
        # new='yes'
    


        
    #     # new='yes'
    #     for s in row[2].split('_'):
    #         if search in s:
    #             new=s
    # #             new= s.split(search)[0]
    # #             # print(new)
    # else:
    #     new='0'
      
    # c.execute(f'UPDATE sheep SET {col_to_update} = ? WHERE id = ?',(new,row[0],))
conn.commit()
conn.close()




In [ ]:
#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','sheep')

numbs=list(range(1,53))
data=db.get_spectra()
print(data)
data=[(tup[0], tup[1], preprocess.process(tup[2])) for tup in data]
# overs=[oversat_check((d[1], d[2])) for d in data]
# print(overs.count(True))
    

# traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]
traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]


fig=go.Figure(traces)
# fig.show()

data

[]


[]